In [11]:
# Values of a healthy heart
V = 5
Rs = 17.5
Rp = 1.79
Kr = 2.8
Kl = 1.12

# Values of a heart with heart fealure
# Rs = 6.82
# Rp = 1.36
# Kr = 4.72
# Kl = 9.5

# Values of a heart with hypertension
# Rs = 40.5
# Rp = 3.21
# Kr = 3
# Kl = 1.7

In [12]:
from sympy import Symbol

# Definition of variables
Csa = Symbol('Csa')
Csv = Symbol('Csv')
Cpa = Symbol('Cpa')
Cpv = Symbol('Cpv')

In [13]:
# Definition of equations
Tsa = Csa/Kr+Csa*Rs
Tsv = Csv/Kr
Tpa = Cpa/Kl+Cpa*Rp
Tpv = Cpv/Kl

Tsum = Tsa+Tsv+Tpa+Tpv

In [14]:
from sympy.utilities.lambdify import lambdify

# Volumes
Vsa = Tsa*V/Tsum
Vsv = Tsv*V/Tsum
Vpa = Tpa*V/Tsum
Vpv = Tpv*V/Tsum
Vsum = Vsa+Vsv + Vpa+Vpv
Vtot = lambdify((Csa,Csv,Cpa,Cpv), 
            Vsum)

In [15]:
# Pressures
Psa = Tsa*V/ (Csa*Tsum)
Psv = Tsv*V/ (Csv*Tsum)
Ppa = Tpa*V/ (Cpa*Tsum)
Ppv = Tpv*V/ (Cpv*Tsum)

In [16]:
from sympy.utilities.lambdify import lambdify
# Objective Function
f_obj = V/(Tsum)
f = lambdify((Csa, Csv, Cpa, Cpv), f_obj)

In [17]:
from sympy.tensor.array import derive_by_array

# Partial derivatives of objective function
partial = derive_by_array(f_obj, (Csa, Csv, Cpa, Cpv))
grad = lambdify((Csa, Csv, Cpa, Cpv), 
            partial)

In [18]:
# Tolerance - num iterations
# tol = 0.001
n = 10000

# Start seed for the random function
# np.random.seed(1)

In [19]:
import numpy as np

# Generate random values for init Cpa, Cpv, Csa, Csv
x = np.random.rand(4)

min_val = 5000000
min_x = x

In [20]:
while(n > 0):
    # Check all x are positive
    for val in x:
        if val < 0:
            val = abs(val)

    # Check the volume restriction
    act_v = Vtot(x[0],x[1],x[2],x[3])

    if act_v != V:
        y = V*np.ones(4)
        diff = np.abs(np.subtract(x,y))
        val, idx = min((val,idx) for (idx,val) in enumerate(diff))
        x[idx] = val

    # Calculate gradient given the points
    g = grad(x[0],x[1],x[2],x[3])

    # Calculate the value of the objective function
    act = f(x[0],x[1],x[2],x[3])

    if(act< min_val):
        min_val = act
        min_x = x

    # Modify the actual value depending the gradient
    val, idx = max((val, idx) for (idx, val) in enumerate(g))
    x[idx] = abs(x[idx]+val)
    n = n - 1

print(' Csa: {} \n Csv: {} \n Cpa: {} \n Cpv: {} \n'.format(
          min_x[0], min_x[1], min_x[2], min_x[3]))


 Csa: 0.7830958927947904 
 Csv: 0.00029997638464297934 
 Cpa: 0.4825289899818529 
 Cpv: 0.2756002171370858 

